In [1]:
#importamos las librerias necesarias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import pickle
import csv

In [2]:
class validation_set:
    
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

In [3]:
class test_set:
    
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

In [4]:
class data_set:
    
    def __init__(self, validation_set, test_set):
        self.validation_set = validation_set
        self.test_set = test_set

In [5]:
def create_csv(name_file, data, col_names, list_opt = False):
    new_data = data.tolist()

    with open(name_file, 'w', newline='') as f:
        if list_opt:
            new_new_data = [[i] for i in new_data]
        else:
            new_new_data = new_data
        write = csv.writer(f)
        write.writerow(col_names)
        write.writerows(new_new_data)

In [6]:
def generate_train_test(file_name):
    df = pd.read_csv(file_name, sep = ',', engine = 'python')
    X = df.drop('RainTomorrow', axis = 1).values
    y = df['RainTomorrow'].values
    columns_names = list(df.columns)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)
    X_columns = columns_names[ :len(columns_names) - 1]
    y_columns = columns_names[len(columns_names) - 1 :]
    return [X_train, y_train, X_test, y_test, X_columns, y_columns]

In [7]:
def generate_folds(data, k):
    X_train = data[0]
    y_train = data[1]
    X_test = data[2]
    y_test = data[3]
    X_columns = data[4]
    y_columns = data[5]
    print('Validación cruzada k =', k)
    validation_sets = []
    kf = KFold(n_splits = k)
    c = 0
    for train_index, test_index in kf.split(X_train):
        c = c + 1
        X_train_v, X_test_v = X_train[train_index], X_train[test_index]
        y_train_v, y_test_v = y_train[train_index], y_train[test_index]
        validation_sets.append(validation_set(X_train_v, y_train_v, X_test_v, y_test_v))
        create_csv(name_file = "./weather/data_validation_train_" + str(k) + "_" + str(c) + ".csv", 
                    data = X_train_v, col_names = X_columns)
        create_csv(name_file = "./weather/data_test_" + str(k) + "_" + str(c) + ".csv", 
                    data = X_test_v, col_names = X_columns)
        create_csv(name_file = "./weather/target_validation_train_" + str(k) + "_" + str(c) + ".csv", 
                    data = y_train_v, col_names = y_columns, list_opt = True)
        create_csv(name_file = "./weather/target_test_" + str(k) + "_" + str(c) + ".csv", 
                    data = y_test_v, col_names = y_columns, list_opt = True)        
    my_test_set = test_set(X_test, y_test)
    my_data_set = data_set(validation_sets, my_test_set) 
    return (my_data_set)

In [9]:
data = generate_train_test('./weatherAUS.csv')
data

[array([['2017-01-19', 'BadgerysCreek', 17.2, ..., 19.4, 21.1, 'No'],
        ['2008-11-20', 'Darwin', 26.2, ..., 30.8, 34.1, 'Yes'],
        ['2014-03-31', 'AliceSprings', 18.1, ..., 29.2, 35.6, 'No'],
        ...,
        ['2014-01-05', 'Nuriootpa', 6.0, ..., 16.1, 16.2, 'No'],
        ['2012-04-26', 'Canberra', 2.5, ..., 9.3, 16.1, 'No'],
        ['2013-12-30', 'Moree', 21.5, ..., 27.4, 38.1, 'No']], dtype=object),
 array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object),
 array([['2012-03-11', 'Perth', 20.2, ..., 28.2, 39.6, 'No'],
        ['2017-03-12', 'Brisbane', 21.5, ..., 27.3, 30.6, 'No'],
        ['2012-10-17', 'Townsville', 16.3, ..., 25.3, 27.4, 'No'],
        ...,
        ['2011-09-17', 'SydneyAirport', 14.5, ..., 18.1, 22.1, 'No'],
        ['2017-03-16', 'Penrith', 20.3, ..., 21.7, 22.2, nan],
        ['2009-01-02', 'Ballarat', 3.0, ..., 9.1, 15.9, 'No']],
       dtype=object),
 array(['No', 'No', 'No', ..., 'No', 'Yes', 'No'], dtype=object),
 ['Date',
  'Location

In [18]:
new_data = generate_folds(data, 3)
dataset_file = open('./weather/dataset_s' + str(3) + '.pkl', 'wb')
pickle.dump(new_data, dataset_file)
dataset_file.close()
print('Completado')

create_csv(name_file = "./weather/data_test.csv", data = new_data.test_set.X_test, col_names = data[4])
create_csv(name_file = "./weather/target_test.csv", data = new_data.test_set.y_test, col_names = data[5], list_opt = True)

Validación Cruzada k = 3
Completado


In [22]:
new_data = generate_folds(data, 5)
dataset_file = open('./weather/dataset_s' + str(5) + '.pkl', 'wb')
pickle.dump(new_data, dataset_file)
dataset_file.close()
print('Completado')

create_csv(name_file = "./weather/data_test.csv", data = new_data.test_set.X_test, col_names = data[4])
create_csv(name_file = "./weather/target_test.csv", data = new_data.test_set.y_test, col_names = data[5], list_opt = True)

Validación Cruzada k = 5
Completado


In [20]:
new_data = generate_folds(data, 10)
dataset_file = open('./weather/dataset_s' + str(10) + '.pkl', 'wb')
pickle.dump(new_data, dataset_file)
dataset_file.close()
print('Completado')

create_csv(name_file = "./weather/data_test.csv", data = new_data.test_set.X_test, col_names = data[4])
create_csv(name_file = "./weather/target_test.csv", data = new_data.test_set.y_test, col_names = data[5], list_opt = True)

Validación Cruzada k = 10
Completado
